In [2]:
import gensim
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import words
from nltk.corpus import wordnet
import nltk
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt

- After loading the data, we clean it

In [3]:
data=pd.read_csv("Emails.csv")
data.fillna(' ',inplace=True)

In [4]:
raw=data['ExtractedSubject'] + ' ' + data['ExtractedBodyText']

### CLEANING

In [5]:
def removeStopWords(String):
    stopwordlist = set(stopwords.words("english"))
    toRemove = ['fm','am',"n't", 'sent','fyi', 'fw', 'pm', 'fwd', 're','cc', 'attachments', "rv", "al", "aq", "la", "ben"]
    stopwordlist.update(toRemove)  
    liste=[word for word in String if word.lower() not in stopwordlist]
    return liste
def removeChar(string):
    return re.sub("[^a-zA-Z]+", " ", string)
def lowerCase(string):
    return string.lower()

In [6]:
i=0
for mail in raw:    
    cleaned_raw=mail
    cleaned_raw=cleaned_raw.lower()
    cleaned_raw=nltk.word_tokenize(cleaned_raw)
    cleaned_raw=removeStopWords(cleaned_raw)
    cleaned_raw=' '.join(cleaned_raw)
    cleaned_raw=removeChar(cleaned_raw)
    cleaned_raw=[token for token in cleaned_raw.split() if len(token)>1]
    raw[i]= cleaned_raw
    i+=1

- We define a dictionary which associate an index to every word
- We also define the corpus in which we will do our analysis

In [8]:
dictionary = gensim.corpora.Dictionary(raw)

In [9]:
corpus = [dictionary.doc2bow(token) for token in raw]

- Using the gensim library, we define the latant dirichlet model

In [20]:
topic=[5,10,20,30,40,50]
topic=[5]
for i in topic:
    model = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=dictionary)
    print("------------------------------------",i, "topics "'------------------------------------')
    print("---------------------------------------------------------------------------------------")
    # arranger l'affichage print(' '.join([w for w, _ in model.show_topic(num_topics=1)]))
    # https://github.com/fdevillard/ada-homeworks/blob/master/05%20-%20Taming%20Text/HW5.ipynb
    print(model.show_topics(num_topics=i))

------------------------------------ 5 topics ------------------------------------
---------------------------------------------------------------------------------------
[(0, '0.005*"women" + 0.005*"would" + 0.004*"said" + 0.003*"us" + 0.003*"time" + 0.003*"speech" + 0.003*"could" + 0.003*"also" + 0.003*"new" + 0.003*"work"'), (1, '0.006*"obama" + 0.005*"state" + 0.005*"would" + 0.005*"new" + 0.004*"one" + 0.004*"israel" + 0.004*"american" + 0.004*"said" + 0.003*"president" + 0.003*"government"'), (2, '0.024*"secretary" + 0.024*"office" + 0.013*"state" + 0.013*"meeting" + 0.010*"room" + 0.009*"department" + 0.007*"house" + 0.007*"arrive" + 0.007*"route" + 0.007*"en"'), (3, '0.005*"would" + 0.004*"call" + 0.004*"one" + 0.003*"new" + 0.003*"said" + 0.003*"also" + 0.003*"us" + 0.003*"president" + 0.003*"people" + 0.003*"obama"'), (4, '0.012*"call" + 0.008*"state" + 0.008*"gov" + 0.005*"get" + 0.005*"today" + 0.005*"see" + 0.004*"know" + 0.004*"would" + 0.004*"good" + 0.004*"cheryl"')]
